# UdaPlay Agent (Starter Style)

This notebook uses `lib.agents.Agent` so the LLM decides which tools to call. Tools are decorated with `@tool` from `lib.tooling`.

In [ ]:
import json
import os
import hashlib
import sys
from pathlib import Path

import chromadb
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient

# Ensure project root is importable for lib/
project_root = Path.cwd()
if not (project_root / "lib").exists():
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from lib.agents import Agent
from lib.tooling import tool, get_tool

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "")
EMBED_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
CHAT_MODEL = os.getenv("OPENAI_CHAT_MODEL", "gpt-4o-mini")

client = OpenAI(api_key=OPENAI_API_KEY)
chroma_client = chromadb.PersistentClient(path=str(project_root / "chroma_db"))
collection = chroma_client.get_or_create_collection(name="udaplay_games")


def embed_texts(texts):
    response = client.embeddings.create(model=EMBED_MODEL, input=texts)
    return [item.embedding for item in response.data]

In [ ]:
QUESTION_KEYWORDS = {
    "release_date": ["when", "release date", "released", "çıktı", "ne zaman"],
    "platforms": ["platform", "hangi platform", "console", "pc", "ps", "xbox", "switch"],
    "developer": ["developer", "developed", "kim geliştirdi", "geliştirici"],
    "publisher": ["publisher", "yayıncı", "published"],
}


def question_needs(query):
    query_lower = query.lower()
    needs = []
    for key, phrases in QUESTION_KEYWORDS.items():
        if any(phrase in query_lower for phrase in phrases):
            needs.append(key)
    return needs

In [ ]:
@tool(
    name="retrieve_game",
    description="Search the local game library (ChromaDB) for relevant info.",
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "User question"},
            "top_k": {"type": "integer", "description": "Number of results"},
        },
        "required": ["query"],
    },
)
def retrieve_game(query: str, top_k: int = 5):
    if not OPENAI_API_KEY:
        return {"error": "Missing OPENAI_API_KEY"}
    query_embedding = embed_texts([query])
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k,
        include=["documents", "metadatas", "distances"],
    )
    matches = []
    for idx, doc in enumerate(results["documents"][0]):
        matches.append(
            {
                "text": doc,
                "metadata": results["metadatas"][0][idx],
                "score": results["distances"][0][idx],
            }
        )
    return matches


@tool(
    name="evaluate_retrieval",
    description="Evaluate if retrieval results are sufficient for answering.",
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string"},
            "results": {"type": "array"},
        },
        "required": ["query", "results"],
    },
)
def evaluate_retrieval(query: str, results: list):
    if not results or isinstance(results, dict) and results.get("error"):
        return {
            "confidence": 0.1,
            "confidence_level": "Low",
            "reason": "No results from the vector store.",
            "needs_web": True,
        }

    needs = question_needs(query)
    satisfied = 0
    for need in needs:
        if any(r.get("metadata", {}).get(need) for r in results):
            satisfied += 1

    satisfied_ratio = 1.0 if not needs else satisfied / len(needs)
    confidence = 0.4 + (0.6 * satisfied_ratio)
    confidence = min(confidence, 0.95)

    if confidence >= 0.75:
        return {
            "confidence": round(confidence, 2),
            "confidence_level": "High",
            "reason": "Results include the key fields needed for the question.",
            "needs_web": False,
        }
    if confidence >= 0.5:
        return {
            "confidence": round(confidence, 2),
            "confidence_level": "Medium",
            "reason": "Results are partially relevant but may miss some fields.",
            "needs_web": True,
        }
    return {
        "confidence": round(confidence, 2),
        "confidence_level": "Low",
        "reason": "Results are weak or missing critical fields.",
        "needs_web": True,
    }


@tool(
    name="game_web_search",
    description="Search the web for game facts and store new info into ChromaDB.",
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string"},
            "max_results": {"type": "integer"},
        },
        "required": ["query"],
    },
)
def game_web_search(query: str, max_results: int = 5):
    if not TAVILY_API_KEY:
        return []

    tavily = TavilyClient(api_key=TAVILY_API_KEY)
    data = tavily.search(query=query, max_results=max_results, include_answer=False)
    sources = []
    for item in data.get("results", []):
        source = {
            "title": item.get("title", ""),
            "url": item.get("url", ""),
            "snippet": item.get("content", ""),
        }
        sources.append(source)

        if OPENAI_API_KEY and source["title"]:
            memory_text = f"{source['title']}\n{source['snippet']}"
            key = source["url"] or source["title"]
            memory_id = "web-" + hashlib.md5(key.encode("utf-8")).hexdigest()
            embedding = embed_texts([memory_text])
            collection.upsert(
                ids=[memory_id],
                documents=[memory_text],
                metadatas=[{"title": source["title"], "source_url": source["url"]}],
                embeddings=embedding,
            )

    return sources

In [ ]:
tools = [get_tool(retrieve_game), get_tool(evaluate_retrieval), get_tool(game_web_search)]

system_prompt = """
You are UdaPlay, a research-style Q&A assistant for video games.
Use tools to answer questions:
1) Call retrieve_game first.
2) Call evaluate_retrieval with the results.
3) If needs_web is true, call game_web_search.

Write in short, clear B1 English.
Return:
- A short answer paragraph.
- A Sources section with citations as title + URL.
- A JSON block with keys: title, release_date, platforms, developer, publisher, sources, confidence.
- A confidence line: High/Medium/Low with a short reason.
"""

agent = Agent(model=CHAT_MODEL, tools=tools, system_prompt=system_prompt, api_key=OPENAI_API_KEY)

questions = [
    "FIFA 21'i kim geliştirdi?",
    "God of War Ragnarok ne zaman çıktı?",
    "Rockstar şu an ne üzerinde çalışıyor?",
]

for q in questions:
    print("=" * 80)
    print("Question:", q)
    result = agent.run(q)

    print("\nTool usage:")
    for message in result["messages"]:
        if message.get("tool_calls"):
            for call in message["tool_calls"]:
                print("-", call["function"]["name"], call["function"]["arguments"])
        if message.get("role") == "tool":
            print("  result:", message["content"][:200], "...")

    print("\nFinal Answer:\n", result["answer"])